In [2]:
import matplotlib.pyplot as plt
from chainercv.datasets import voc_bbox_label_names
from chainercv.links import FasterRCNNVGG16, SSD300, SSD512, YOLOv3, YOLOv2
from chainercv.utils import read_image
from PIL import Image
import os

model0 = FasterRCNNVGG16(pretrained_model='voc07')
model1 = SSD300(pretrained_model='voc0712')
model2 = SSD512(pretrained_model='voc0712')
model3 = YOLOv3(pretrained_model='voc0712')
model4 = YOLOv2(pretrained_model='voc0712')

In [3]:
def save_img(bboxes,full_path):
    img = Image.open(full_path)
    top, left , bottom, right  = bboxes[0][0]
    cropped = img.crop( ( left, top, right, bottom) )
    try:
        cropped.save(full_path, "JPEG", quality=80, optimize=True, progressive=True)
    except IOError:
        PIL.ImageFile.MAXBLOCK = cropped.size[0] * cropped.size[1]
        cropped.save(full_path, "JPEG", quality=80, optimize=True, progressive=True)

In [4]:
def comparer_model(model, img):
    bboxes0, labels0, scores0 = model.predict([img])
    if len(bboxes0[0])>0:
        if labels0[0][0]==2:
            maximum_proba=scores0[0][0]
            return maximum_proba, bboxes0
    return -1,-1

In [6]:
def main_crop(path):
    for dossier, sous_dossiers, fichiers in os.walk(path):
        print (sous_dossiers)
        for num, fichier in enumerate(fichiers):
            if num%10==0:
                print(num)
            full_path = os.path.join(dossier, fichier)
            img = read_image(full_path)
            maximum0, bboxes_f0 = comparer_model(model0, img)
            maximum1, bboxes_f1 = comparer_model(model1, img)
            maximum2, bboxes_f2 = comparer_model(model2, img)
            maximum3, bboxes_f3 = comparer_model(model3, img)
            maximum4, bboxes_f4 = comparer_model(model4, img)
            
            boxes = [bboxes_f0, bboxes_f1, bboxes_f2, bboxes_f3, bboxes_f4]
            maxes = [maximum0, maximum1, maximum2, maximum3, maximum4]
            
            if max(maxes)<=0.9:
                continue
            box = boxes[boxes.index(max(boxes))]
            save_img(box, full_path)

In [74]:
def create_file_label_map(path, dest_name):
    data_type = path.split('/')[-1]
    with open(dest_name, 'w') as dest:
        for folder, sub_folders, all_files in os.walk(path):
            class_count = 0
            for sub_folder in sub_folders:
#                 print(path+"/"+sub_folder)
                walk = os.walk(path+"/"+sub_folder)
                for infos in walk:
                    sub_files = infos[2]
                    for idx, file in enumerate(sub_files):
#                         print(file)
                        file_path = os.path.join(data_type+"/"+sub_folder, file)
                        dest.write(file_path + ":" + str(class_count) + "\n")
                    class_count += 1

In [76]:
create_file_label_map('./bird_dataset_crop/train_images', './bird_dataset_crop/train.txt')
create_file_label_map('./bird_dataset_crop/val_images', './bird_dataset_crop/val.txt')
create_file_label_map('./bird_dataset_crop/test_images', './bird_dataset_crop/test.txt')

create_file_label_map('./bird_dataset_mix/train_images', './bird_dataset_mix/train.txt')
create_file_label_map('./bird_dataset_mix/val_images', './bird_dataset_mix/val.txt')
create_file_label_map('./bird_dataset_mix/test_images', './bird_dataset_mix/test.txt')

create_file_label_map('./bird_dataset/train_images', './bird_dataset/train.txt')
create_file_label_map('./bird_dataset/val_images', './bird_dataset/val.txt')
create_file_label_map('./bird_dataset/test_images', './bird_dataset/test.txt')

In [7]:
main_crop("./bird_dataset2/test_images")
print("Done Test")
main_crop("./bird_dataset2/train_images")
print("Done Train")
main_crop("./bird_dataset2/val_images")
print("Done Val")

['mistery_category']
[]
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
['021.Eastern_Towhee', '016.Painted_Bunting', '015.Lazuli_Bunting', '033.Yellow_billed_Cuckoo', '028.Brown_Creeper', '012.Yellow_headed_Blackbird', '011.Rusty_Blackbird', '009.Brewer_Blackbird', '026.Bronzed_Cowbird', '034.Gray_crowned_Rosy_Finch', '029.American_Crow', '013.Bobolink', '020.Yellow_breasted_Chat', '023.Brandt_Cormorant', '004.Groove_billed_Ani', '019.Gray_Catbird', '010.Red_winged_Blackbird', '030.Fish_Crow', '031.Black_billed_Cuckoo', '014.Indigo_Bunting']
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]
0
10
20
30
40
50
[]

In [20]:
import os

target_dir = "./bird_dataset2/train_images"

for path, dirs, files in os.walk(target_dir):
    for file in files:
        new_path = path.split("/")
        new_path[1] = 'bird_dataset3'
        new_path = '/'.join(new_path)
        filename, ext = os.path.splitext(file)
        new_file = filename + "_crop.jpg"
        old_filepath = os.path.join(path, file)
        new_filepath = os.path.join(new_path, new_file)
        os.rename(old_filepath, new_filepath)